In [1]:
import time

import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
from sklearn.metrics import r2_score

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 16)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 12})

print("Package is ready.")

Package is ready.


In [ ]:
# Load them!
l = np.load("./Data/ECtrainset_wRWEC.npz")

# See what's in here
print (l.files)

# get data
train_data = l['traindataset']
train_label = l['trainlabel']
test_data = l['testdataset']
test_label = l['testlabel']

mins = l['mins']
maxs = l['maxs']

labelmins = l['labelmins']
labelmaxs = l['labelmaxs']

explanation = "RWEC_LSTM_24"
DELETE = [3, 4, 5, 11, 12]
EX_PRED = 0 #Sorry, it's not automated yet.

train_data = np.delete(train_data, DELETE, axis=1)
test_data = np.delete(test_data, DELETE, axis=1)
train_label = train_label[:,:]
test_label = test_label[:,:]

print(train_data[:5])
print(train_label[:5])
print(test_data[:5])
print(test_label[:5])

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

In [ ]:
#Data setting
n_train = train_label.shape[0]
n_test = test_label.shape[0]
n_input = train_data.shape[1]
n_output = test_label.shape[1]
n_classes = n_output

print("# train: %d # test: %d, # input: %d, # output: %d" % (n_train, n_test, n_input, n_output))


In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 10000
batch_size = n_train
display_step = 500

# Network Parameters
n_steps = 10 # 1.5 day
n_hidden = 64 # hidden layer num of features

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_output])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_output]))
}

print("parameters are ready")

In [ ]:
# Load them!
l = np.load("./Data/ECpredset_wRWEC.npz")

# See what's in here
print (l.files)

# get data
pred_data = l['preddataset']
pred_label = l['predlabel']
n_pred = pred_label.shape[0]

print(pred_data[0])
print(pred_label[0])

print(pred_data.shape)
print(pred_label.shape)

#Processing

for i in range(pred_data.shape[1]):
    temp = pred_data[:,i]
    temp = (temp - mins[i]) / (maxs[i] - mins[i])
    pred_data[:,i] = temp

    
for i in range(pred_label.shape[1]):
    temp = pred_label[:,i]
    temp = (temp - labelmins[i]) / (labelmaxs[i] - labelmins[i])
    pred_label[:,i] = temp
    
pred_data = np.delete(pred_data, DELETE, axis=1)

pred_x = pred_data.reshape((-1, n_steps, n_input))

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

print("parameters are ready")

In [ ]:
# Define loss and optimizer
cost = tf.multiply(tf.reduce_sum(tf.square(tf.subtract(pred, y))),1/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
ymean = tf.reduce_mean(y)
SSE = tf.reduce_sum(tf.square(tf.subtract(y, pred)))
SSR = tf.reduce_sum(tf.square(tf.subtract(pred, ymean)))
r_squared = SSR/(SSE+SSR)
                                
accuracy = r_squared
# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Do some optimizations
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(init)

print("seesion start")

In [ ]:
# Summary writer
tf.summary.scalar('RMS', cost)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()
directory_name = "./tb_logs/" + explanation + str(n_hidden)
summary_writer = tf.summary.FileWriter(directory_name, graph=sess.graph)
print ("Summary ready")

In [ ]:
train_data.shape

In [ ]:
num_data = train_data.shape[0]
int(num_data/batch_size/n_steps)

In [ ]:
batch_x = train_data
batch_x.shape

In [ ]:
batch_x = batch_x.reshape((batch_size, n_steps, n_input))
batch_x.shape

In [ ]:
batch_y = train_label[i:i+batch_size, :]
batch_y.shape

In [ ]:
# Launch the graph
num_data = train_data.shape[0]
ex_pred_acc = EX_PRED

# Keep training until reach max iterations
start = time.time()

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(num_data/batch_size/n_steps)

    for i in range(total_batch):
        batch_x = train_data[i:i+n_steps*batch_size, :]
        batch_y = train_label[i:i+batch_size, :]
        
        # Reshape data
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        summary, _ = sess.run([merged, optimizer], feed_dict={x: batch_x, y: batch_y})
        avg_cost += sess.run(cost, feed_dict={x: batch_x, y: batch_y})/total_batch
        summary_writer.add_summary(summary, epoch*total_batch+i)
        
        pred_acc = sess.run(accuracy, feed_dict={x: pred_x, y: pred_label})

    if pred_acc > ex_pred_acc:
        bestpath = "./models/best_trained_" + explanation + ".ckpt"
        save_path = saver.save(sess, bestpath)
        ex_pred_acc = pred_acc
        
    if epoch % display_step == 0:
        print ("Epoch: %04d/%04d cost: %.6f, best: %.3f" % (epoch, training_epochs, avg_cost,ex_pred_acc))
        train_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})

        test_x = test_data.reshape((-1, n_steps, n_input))
        test_acc = sess.run(accuracy, feed_dict={x: test_x, y: test_label})
        pred_acc = sess.run(accuracy, feed_dict={x: pred_x, y: pred_label})
        print ("Training Acc: %.3f, Validation Acc: %.3f, Test Acc: %.3f" % (train_acc, test_acc, pred_acc))
        
            
print ("Epoch: %04d/%04d cost: %.6f" % (epoch+1, training_epochs, avg_cost))
train_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})

test_x = test_data.reshape((-1, n_steps, n_input))
test_acc = sess.run(accuracy, feed_dict={x: test_x, y: test_label})
print ("Training Acc: %.3f, Validation Acc: %.3f, Test Acc: %.3f" % (train_acc, test_acc, pred_acc))

end = time.time() - start
print ("Optimization Finished\ntraining time: %.2f sec." % (end))

In [ ]:
#Choose best model
bestpath = "./models/best_trained_" + explanation + ".ckpt"
saver.restore(sess, bestpath)
# print("Model restored, best Acc: %.3f" % ex_pred_acc)

In [ ]:
#Calculate accuracy for 128 mnist test images
print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: pred_x, y: pred_label}))

In [ ]:
estimation = sess.run(pred, feed_dict={x: test_x, y: test_label})
estimation.shape

In [ ]:
#Data restoring
for i in range(n_output):
    estimation[:,i] = (labelmaxs[i] - labelmins[i]) * estimation[:,i] + labelmins[i]
    test_label[:,i] = (labelmaxs[i] - labelmins[i]) * test_label[:,i] + labelmins[i]

print(estimation[:5])
print(test_label[:5])

In [ ]:
valid_RMSE = np.sum(np.sqrt((test_label - estimation)**2))/(estimation.shape[0]*18)

print("valid RMSE: %.5f" % valid_RMSE)

In [ ]:
for i in range(test_label.shape[1]):
    print(i+1, r2_score(test_label[:, i], estimation[:, i]))

In [ ]:
#1:1 graph
x_t = np.arange(-5, 10)
y_t = x_t

xny = plt.plot(x_t,y_t,'k')

expect1 = plt.plot(test_label, estimation, 'b.')

plt.axis([2, 6, 2, 6])
plt.grid(True)
plt.xlabel('Measured EC', {'fontsize':20})
plt.ylabel('Estimated EC',  {'fontsize':20})
#plt.legend([expect1], ["Mg"], prop={'size': 12})
plt.show()

In [ ]:
regression_output = np.concatenate((test_label.T, estimation.T), axis=0).T

file_name = "regression_output_" + explanation + ".CSV"
np.savetxt("./Results/" + file_name, regression_output, delimiter=",")

In [ ]:
#chronological graph
x_t = np.arange(0, n_output)

measured1 = plt.plot(x_t, test_label[55,:].T, 'k') #estimated

expect1 = plt.plot(x_t, estimation[55,:].T, 'b') #estimated



plt.axis([0, n_output-1, 3, 4])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
#chronological graph
i = 72
x_t = np.arange(0, n_output)

measured1 = plt.plot(x_t, test_label[i,:].T, 'k') #estimated

expect1 = plt.plot(x_t, estimation[i,:].T, 'b') #estimated



plt.axis([0, n_output-1, 4, 5])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
#chronological graph
x_t = np.arange(0, n_output)

measured1 = plt.plot(x_t, test_label[:20,:].T, 'k') #estimated
expect1 = plt.plot(x_t, estimation[:20,:].T, 'b') #estimated

plt.axis([0, n_output-1, 2, 6])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
# Load them!
l = np.load("./Data/ECpredset_wRWEC.npz")

# See what's in here
print (l.files)

# get data
pred_data = l['preddataset']
pred_label = l['predlabel']
n_pred = pred_label.shape[0]

print(pred_data[0])
print(pred_label[0])

print(pred_data.shape)
print(pred_label.shape)

In [ ]:
#Processing

for i in range(pred_data.shape[1]):
    temp = pred_data[:,i]
    temp = (temp - mins[i]) / (maxs[i] - mins[i])
    pred_data[:,i] = temp

    
for i in range(pred_label.shape[1]):
    temp = pred_label[:,i]
    temp = (temp - labelmins[i]) / (labelmaxs[i] - labelmins[i])
    pred_label[:,i] = temp

In [ ]:
pred_data = np.delete(pred_data, DELETE, axis=1)

In [ ]:
print(pred_data[0])
print(pred_label[0])

In [ ]:
pred_data.shape

In [ ]:
pred_x = pred_data.reshape((-1, n_steps, n_input))

print ("Prediction Accuracy:", sess.run(accuracy, feed_dict={x: pred_x, y: pred_label}))
prediction = sess.run(pred, feed_dict={x: pred_x, y: pred_label})

In [ ]:
#Data restoring
for i in range(n_output):
    prediction[:,i] = (labelmaxs[i] - labelmins[i]) * prediction[:,i] + labelmins[i]
    pred_label[:,i] = (labelmaxs[i] - labelmins[i]) * pred_label[:,i] + labelmins[i]

print(prediction[0])
print(pred_label[0])

In [ ]:
test_RMSE = np.sum(np.sqrt((pred_label - prediction)**2))/(prediction.shape[0]*18)

print("test RMSE: %.5f" % test_RMSE)

In [ ]:
#1:1 graph
x_t = np.arange(-5, 15)
y_t = x_t

xny = plt.plot(x_t,y_t,'k')

expect1 = plt.plot(pred_label, prediction, 'b.')

plt.axis([4, 5.5, 4, 5.5])
plt.grid(True)
plt.xlabel('Measured EC', {'fontsize':20})
plt.ylabel('Estimated EC',  {'fontsize':20})
#plt.legend([expect1], ["Mg"], prop={'size': 12})
plt.show()

In [ ]:
RMSE_data = []
for i in range(pred_label.shape[1]):
    k = np.sum(np.sqrt((pred_label[:, i] - prediction[:, i])**2))/120
    print(i+1, k)
    RMSE_data.append(k)
print(RMSE_data)


x_t = np.arange(0, 18)
y_t = x_t*0 + test_RMSE
y2_t = x_t*0 + valid_RMSE

RMSEs = plt.plot(x_t,RMSE_data, 'k')
test_RMSE = plt.plot(x_t, y_t, 'k--')
valid_RMSE = plt.plot(x_t, y2_t, 'k-')

plt.axis([0, 17, 0, 0.2])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
#chronological graph
i = 0
j = 11
k = 72
x_t = np.arange(0, 18)


expect1 = plt.plot(x_t, prediction[i,:].T, 'b') #estimated
measured1 = plt.plot(x_t, pred_label[i,:].T, 'b--') #estimated

expect2 = plt.plot(x_t, prediction[j,:].T, 'r') #estimated
measured2 = plt.plot(x_t, pred_label[j,:].T, 'r--') #estimated

expect3 = plt.plot(x_t, prediction[k,:].T, 'g') #estimated
measured3 = plt.plot(x_t, pred_label[k,:].T, 'g--') #estimated


plt.axis([0, 17, 4.2, 5.2])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
#chronological graph
x_t = np.arange(0, 18)

#measured1 = plt.plot(x_t, pred_label[:,:].T, 'k') #estimated
expect1 = plt.plot(x_t, prediction[:15,:].T, 'b') #estimated

plt.axis([0, 17, 4, 5.25])
plt.grid(True)
plt.xlabel('Time (h)', {'fontsize':20})
plt.ylabel('EC',  {'fontsize':20})
plt.show()

In [ ]:
regression_output = np.concatenate((pred_label.T, prediction.T), axis=0).T

file_name = "pred_output_" + explanation + ".CSV"
np.savetxt("./Results/" + file_name, regression_output, delimiter=",")

In [ ]:
sess.close()